In [97]:
from collections import defaultdict
from functools import reduce
class Node:
    def __init__(self):
        self.state = 0
        self.input_history = defaultdict(int)
        self.destinations = []

class Board:
    def __init__(self):
        self.broadcast = [] #List of destination nodes
        self.flip_flop = defaultdict(Node)
        self.conjunction = defaultdict(Node)


In [40]:
def read_file(filename='test'):
    b = Board()
    input_file = open(f'data/{filename}_20', 'r')
    data = input_file.read().splitlines()
    for n in data:
        node_name, destinations = n.split(' -> ')
        if node_name == 'broadcaster':
            b.broadcast = destinations.split(', ')
        elif node_name.startswith("%"):
            b.flip_flop[node_name[1:]].destinations = destinations.split(', ')
        else:
            b.conjunction[node_name[1:]].destinations = destinations.split(', ')
    for n in b.flip_flop:
        for d in b.flip_flop[n].destinations:
            if d in b.conjunction:
                b.conjunction[d].input_history[n] = 0
    for n in b.conjunction:
        for d in b.conjunction[n].destinations:
            if d in b.conjunction:
                b.conjunction[d].input_history[n] = 0
    return b

In [66]:
def push_button(b, pulse_counter):
    pulses = []
    pulses.append(('button', 'broadcaster', 0))
    while pulses:
        current_pulse = pulses.pop(0)
        from_node, node_name, pulse = current_pulse
        pulse_counter[pulse] += 1
        if node_name == 'broadcaster':
            for n in b.broadcast:
                pulses.append((node_name, n, pulse))
        elif node_name in b.flip_flop:
            if pulse == 1:
                continue
            else:
                b.flip_flop[node_name].state = (b.flip_flop[node_name].state + 1) % 2
                for n in b.flip_flop[node_name].destinations:
                    pulses.append((node_name, n, b.flip_flop[node_name].state))
        else:
            b.conjunction[node_name].input_history[from_node] = pulse
            new_pulse = (int(all(b.conjunction[node_name].input_history.values())) + 1) % 2
            for n in b.conjunction[node_name].destinations:
                pulses.append((node_name, n, new_pulse))
    return (b, pulse_counter)

In [89]:
def push_button_earlystop(b, cancel_node_name = 'rx', cancel_signal = 0, cancel_signal_from_node = None):
    pulses = []
    pulses.append(('button', 'broadcaster', 0))
    while pulses:
        current_pulse = pulses.pop(0)
        from_node, node_name, pulse = current_pulse
        if cancel_signal_from_node == None:
            cancel_signal_from_node = from_node
        if (node_name == cancel_node_name) and (pulse == cancel_signal) and (from_node == cancel_signal_from_node):
            return (b, True)
        if node_name == 'broadcaster':
            for n in b.broadcast:
                pulses.append((node_name, n, pulse))
        elif node_name in b.flip_flop:
            if pulse == 1:
                continue
            else:
                b.flip_flop[node_name].state = (b.flip_flop[node_name].state + 1) % 2
                for n in b.flip_flop[node_name].destinations:
                    pulses.append((node_name, n, b.flip_flop[node_name].state))
        else:
            b.conjunction[node_name].input_history[from_node] = pulse
            new_pulse = (int(all(b.conjunction[node_name].input_history.values())) + 1) % 2
            for n in b.conjunction[node_name].destinations:
                pulses.append((node_name, n, new_pulse))
    return (b, False)

In [100]:
def first_star(filename='test'):
    b = read_file('input')
    pulse_counter = defaultdict(int)

    for i in range(1000):
        b, pulse_counter = push_button(b, pulse_counter)

    return pulse_counter[0]*pulse_counter[1]

first_star('input')

680278040

In [85]:
#Brute Force - takes too long time
b = read_file('input')
for i in range(1000000000000):
    b, res = push_button_earlystop(b, cancel_node_name = 'rx', cancel_signal = 0)
    if res:
        print(i)
        break

KeyboardInterrupt: 

# Reverse Engineering

## General info from input file
RX gets signal from &vf
VF gets signal from &hf, &pm, &mk, &pk

## Chain of thought
So RX gets a LOW signal only when all inputs from VF sent recently HIGH signal.
There are 4 inputs to VF, which are also conjunction. We need to find separate the button clicks number, when those 4 send HIGH signal.

In [101]:
def second_star(filename='test'):
    res = {}
    for n in ['hf','pm', 'mk', 'pk']:
        b = read_file('input')
        for i in range(1000000000):
            b, flound = push_button_earlystop(b, cancel_node_name = 'vf', cancel_signal = 1, cancel_signal_from_node=n)
            if flound:
                res[n] = (i+1)
                break
    return reduce((lambda x, y: x * y), res.values())

second_star('input')

243548140870057